In [ ]:
%pylab inline

import cosima_cookbook as cc
#session = cc.database.create_session()
import matplotlib.pyplot as plt # to generate plots
from mpl_toolkits.basemap import Basemap # plot on map projections
from glob import glob
import xarray as xr
import scipy as sp
import pandas as pd
import scipy.interpolate
import scipy.ndimage
from tqdm import tqdm_notebook
import IPython.display
import cmocean as cm
import cartopy.crs as ccrs
import cartopy.feature as cft
from dask.distributed import Client, progress

import sys, os
sys.path.append(os.path.join(os.getcwd(), '..'))  # so we can import ../exptdata
import exptdata
print('Available exptdata keys: ', [k for k in exptdata.exptdict.keys()])

In [ ]:
cc.start_cluster()
#client = Client('tcp://10.0.64.4:8786',local_dir='/g/data/e14/ss2778/tmp')
#client

In [ ]:
aviso_clim_tstart = pd.to_datetime('1958', format='%Y')
aviso_clim_tend = aviso_clim_tstart + pd.DateOffset(years=60)
firstyear = pd.to_datetime(aviso_clim_tstart).year  # assumes tstart is 1 January!
lastyear = pd.to_datetime(aviso_clim_tend).year-1  # assumes tend is 1 January!
yearrange = str(firstyear)+'-'+str(lastyear)
print(yearrange)
print('aviso_clim_tstart = ', aviso_clim_tstart)
print('aviso_clim_tend = ', aviso_clim_tend)

In [ ]:
figdir = ''
def savefigure(fname):
    plt.savefig(os.path.join(figdir, fname+'.png'),dpi=300, bbox_inches="tight")  # comment out to disable saving
    #plt.savefig(os.path.join(figdir, fname+'.pdf'),dpi=300, bbox_inches="tight")  # comment out to disable saving
    return

In [ ]:
ekey='025deg'
expt = exptdata.exptdict[ekey]['expt']
n_files = exptdata.exptdict[ekey]['n_files']
time_units = exptdata.exptdict[ekey]['time_units']
offset = exptdata.exptdict[ekey]['offset']
    
sl = cc.get_nc_variable(expt,'ocean_month.nc','sea_level',
                                 n=n_files,time_units=time_units, offset=offset,use_cache=True)\
                                 .sel(time=slice(aviso_clim_tstart,aviso_clim_tend)).load()
sl_sq = cc.get_nc_variable(expt,'ocean_month.nc','sea_levelsq',
                             n=n_files,time_units=time_units, offset=offset,use_cache=True)\
                             .sel(time=slice(aviso_clim_tstart,aviso_clim_tend)).load()
    
SSTm = cc.get_nc_variable(expt,'ocean_month.nc','surface_temp',
                                 n=n_files,time_units=time_units, offset=offset,use_cache=True)\
                                 .sel(time=slice(aviso_clim_tstart,aviso_clim_tend))- 273.15
SSTm.load();
#sea_level = cc.querying.getvar(expt=expt,variable='sea_level',session=session,ncfile='ocean_month.nc',time_units=time_units,n=n_files,offset=offset)

In [ ]:
x=sl.rolling(time=6,center=True).mean();
y=sl_sq.rolling(time=6,center=True).mean();
var_six= (y - x**(2.0));
var_six.load();

In [ ]:
var_sl=var_six[3:298,:,:] #Removing Nan from the data
#De-sesonalized the variable:
Clim_sl=var_sl.groupby('time.month').mean('time')
out_sl=var_sl.groupby('time.month')
var_des=(out_sl-Clim_sl)

# Nino 3.4 time series

In [ ]:
# Load Nino 3.4 time series:
nino= SSTm.sel(xt_ocean=slice(-170,-120)).sel(yt_ocean=slice(-5,5))
SST_avg=nino.mean('xt_ocean').mean('yt_ocean')
Clim=SST_avg.groupby('time.month').mean('time')
out=SST_avg.groupby('time.month')
SSTa=(out-Clim)
#series=SSTa.to_pandas()
Nino_nor= (SSTa-np.mean(SSTa))/np.std(SSTa)

In [ ]:
sstn=Nino_nor[3:298] #Making time dimension same as Variance of sea_level
tl=len(var_des.time)
#Calculate Nino 3.4 regressed variance of Sea_level:
reg_sl=np.dot(sstn,np.transpose(var_des.values,(1,0,2)))/tl

In [ ]:
#Regression plot
plt.figure(figsize=(15,8))
X,Y=np.meshgrid(var_des.xt_ocean,var_des.yt_ocean)
h=plt.pcolormesh(X,Y,reg_sl*1000.,cmap='RdBu_r',vmax= 0.5,vmin=-0.5)
cb = plt.colorbar(h,orientation='vertical',label='cm2/std')
#plt.contour(X,Y,reg,levels=5, linewidths=0.5, colors='k')
plt.title('JRA55-do Nino 3.4 regressed variance of sea_level')
plt.ylabel('latitude(degrees)')
plt.xlabel('longitude(degrees)')
savefigure('Regressed variance of sea_level_rm-6c_25y_1deg')

In [ ]:
n=len(var_des.yt_ocean)
m=len(var_des.xt_ocean)
error = np.empty((t, 300, 360))
SE =np.empty((300,360))
rms=np.empty((300,360))
for i in range(1,t): # loop over time dimension
    error[i,:,:] = var_des[i,:,:] - np.dot(reg_sl[i,:,:],sstn[i]) # regression here (other way)
    if i%100 == 0: # modulo function
        print(i) # print every 10th iteration
de= xr.Dataset({ 'error': (('time', 'latitude', 'longitude'), error)})
for i in range(1,n):
    for j in range(1,m): #loop over lat and lon
        SE[i,j] = ((np.var(error[:,i,j],axis=0))/sqrt(t-1))
        rms[i,j]=np.sqrt(SE[i,j])
    if i%100 == 0:# modulo function
        if j%100 == 0:# modulo function
            print(i,j) 
dse= xr.Dataset({'SE': (('latitude', 'longitude'), SE),'rms': (('latitude','longitude'),rms)})